# 2.4 通过微调，提升模型NL2SQL准确度

## 🚅 前言

模型微调是一种可选的提高大模型在特定场景表现的一种手段。

**知识库索引、插件调用**等类似于考生应对开卷考试的过程，模型不需要提前学习特定知识，只需要根据额外携带的参考书、计算器等外部工具辅助进行推理。

**微调**则类似于考生应对闭卷考试的过程，模型学习微调数据集中特定场景的知识，更新内部参数，使得模型在特定场景中不需要额外知识（如提示词，样例）就可以生成期望的结果。

后者的成本更高、过程更长，但它可以提高模型推理效率，降低调用成本。


## 🍁 课程目标

学完本课程后，你将能够：

*   了解什么是模型微调
    
*   了解构建微调数据集的技巧
    
*   了解如何评测微调后的模型
    
*   通过案例，一步步体验如何微调
    


## 📖 课程目录

- [1. 什么是模型微调](#🤔-1-什么是模型微调)
    - [1.1. 微调的优点](#11-微调的优点)
    - [1.2. 微调的前提](#12-微调的前提)
    - [1.3. 微调的基本步骤](#13-微调的基本步骤)
- [2. 微调的常用算法](#🧠-2-微调的常用算法)
    - [2.1. 全参微调](#21-全参微调)
    - [2.2. 高效微调](#22-高效微调)
- [3. 微调数据集的构建](#🧬-3-微调数据集的构建)
    - [3.1. 微调数据格式](#31-微调数据格式)
    - [3.2. 数据集的规模要求](#32-数据集的规模要求)
    - [3.3. 构建高效微调数据集的策略](#33-构建高效微调数据集的策略)
    - [3.4. 训练集、验证集、测试集拆分](#34-训练集、验证集、测试集拆分)
- [4. 一个完整的微调实践](#🛠️-4-一个完整的微调实践)
    - [4.1. 实验环境准备](#41-实验环境准备)
    - [4.2. 把模型下载到本地（可选）](#42-把模型下载到本地（可选）)
    - [4.3. 准备微调数据集](#43-准备微调数据集)
    - [4.4. 使用预训练模型](#44-使用预训练模型)
    - [4.5. 评测预训练模型性能](#45-评测预训练模型性能)
    - [4.6. 微调参数说明](#46-微调参数说明)
    - [4.7. 快速估模型训练的效果](#47-快速估模型训练的效果)
    - [4.8. 使用微调后模型](#48-使用微调后模型)
    - [4.9. 参数矩阵融合（可选）](#49-参数矩阵融合（可选）)
    - [4.10. 评测微调后的模型](#410-评测微调后的模型)



## 🤔 1. 什么是模型微调

模型微调，或称为Fine-tuning，是一种机器学习技术，尤其在深度学习领域广泛应用。该技术基于一个预训练模型（pre-trained model）—— 一个已经在大规模数据集上经过训练、学习了通用知识的模型。在这个预训练模型上通过在特定任务的数据集上进行额外的训练，这样不仅可以利用预训练模型的泛化能力，还能让模型学会解决特定问题的细微差别，从而在保持模型相对轻量化的同时，显著提升在特定任务上的准确度和效率。

### 1.1. 微调的优点

1.  **提高模型适应性**：针对特定任务的微调使得模型能够更好地理解和处理该领域的特定语言结构和专业术语。
    
2.  **轻量级模型的高性能**：相比直接使用庞大且复杂的模型，微调可以使较小的模型上在特定任务有更好表现，有助于降低推理延迟，提高用户体验。
    
3.  **抑制模型幻觉：** 通过在真实世界的、针对性的数据集上进行微调，模型可以学习到更加精确和可靠的上下文关联，减少生成内容中的误导信息和逻辑错误，增强输出的合理性和实用性。
   

### 1.2. 微调的前提

*   **业务需求匹配度**

    *   微调前需要明确业务的具体需求，明确要通过大模型微调解决的具体业务问题和应用场景。可以先问自己以下问题：
        
    *   **你的任务是否需要复杂的语言理解或生成能力？** 例如，复杂的自然语言生成、意图识别等任务可能受益于大模型微调。
        
    *   **你的任务是否需要高度特定于某个领域或任务的语言能力？** 例如，法律文书分析、医学文本理解等领域任务。
        
    *   当前的模型是否已能满足大部分需求？如果能满足，则可能不需要微调。
        
    *   **是否有具体的业务指标来衡量微调前后效果对比？** 比如微调后的大模型推送给客户的信息更加准确，从而降低投诉率。
    

*   **尝试更大的模型**

    随着大模型的发展，现在模型能力越来越强，如果你只是想改善模型性能，尝试使用参数更大的模型往往能带来非常直观的性能提升，因为参数更大的模型经历了更广泛的数据集和更多轮次的训练。  
    当然，如果你有诸如使用成本、模型延迟等限制，也仍然建议使用更大的模型进行实验或者性能验证。因为更大的模型在微调时能做为“老师”，通过生产微调数据的方式，将自己的知识传授给自己的“学生”。
    
*   **使用提示工程、插件调用。**

    相比于模型微调，提示工程、插件调用等辅助增强模型输入的手段是一种更灵活的选择。因为：
    
    *   在许多任务中，模型最初可能表现不佳，但通过应用正确的 Prompt 技巧可以改进结果，不一定需要使用模型调优。
        
    *   迭代优化 Prompt、插件，比模型调优的迭代更敏捷、成本更低，因为模型调优的迭代可能需要重新收集数据、清洗优化数据、收集 bad case、发起客户调研等。
        
    *   即使最后一定要进行模型调优，最初的 Prompt 工程、插件迭代优化相关工作也不会浪费。你的这些前期工作可以充分地在构建调优数据集时复用（用于构建数据集的输入）。
        
*   **资源和技术可行性**

    微调过程需要计算资源和时间成本，包括GPU资源、存储空间以及可能的**专家人力成本**。需要评估项目预算和资源是否允许进行有效微调。同时也要评估团队是否具备微调大模型所需的技术能力和经验，或者是否有合适的合作伙伴提供技术支持。以 Llama3 的微调为例，最大的70B 模型在 lora 微调下大约需要  2 块 H100 GPU（单卡80GB显存），存储空间 TB 级别可以完成（取决于微调数据量），微调的运行时间大概在 200 小时左右。
    
*   **需要足够多的数据**

    虽然相对预训练，微调已经对训练数据要求非常低了，但在实践中仍然需要一个不小的精标数据集，最少需要 1000 条数据才能在有良好的效果。
    

*   **合规与隐私**

      业务工作者需要确保使用的数据符合法律法规要求，处理个人数据时遵循隐私保护原则，尤其是GDPR等国际和地区隐私法规。此外，还要关注模型偏见、过拟合、泛化能力不足等潜在风险，以及这些风险对业务可能造成的影响。
    

总的来说，对微调业务的价值进行投资回报率（ROI）分析至关重要，这意味着要进行全面的成本与收益评估，以确定微调措施所带来的商业价值是否超越其成本投入。这一分析不仅应涵盖直接的经济收益，还应考虑诸如用户体验的改善、品牌影响力的增强等间接效益。


### 1.3. 微调的基本步骤

1.  **选择预训练模型**：根据任务需求选择一个合适的预训练模型，如Llama、千问、Bert等，这些开源模型已经在大规模文本数据上进行了广泛训练。
    
2.  **准备微调数据集**：收集或构建与目标任务紧密相关的数据集，确保数据质量高且代表性强。
    
3.  **选择微调方法**：根据任务需求与可用硬件配置选择不同的模型微调方法，如LoRA、QLoRA、 p-Tuning等，设置相应的微调参数。
    
4.  **训练**：在微调数据集上进行训练，通常需要监控过拟合，并可能使用适当的退火策略或正则化技术。
    
5.  **评估与测试**：使用验证集评估模型性能，调整超参数，最终在测试集上验证微调的成果。

## 🧠 2. 微调的常用算法

大模型具有海量参数，在预训练阶段需要海量训练样本，而且需要大量的时间与算力。那么微调的“微”字体现在哪里呢？

“微”字主要体现在以下几个方面：

1.  数据规模小：微调的数据集长度（10^5）往往比预训练的数据集长度（10^12）小很多量级。
    
2.  硬件要求低：由于使用了很多种高效微调（PEFT）策略，模型训练时占用内存和 GPU 性能要求比预训练低的多。比如预训练 Llama3 最少需要 2000块，80GB显存 NVIDIA H100 GPU，而微调 Llama3 只需要 2 块 H100 GPU。
    
3.  训练时间较短：由于预训练前的大模型参数是随机初始化的，因此往往需要很长的训练轮次与时间才能达到预期效果；而微调前的大模型已经经过了预训练这一步骤，微调的时间无需过长即可达到较高的性能。仍然以 Llama3 为例：预训练 Llama3 一轮需要最少需要700个小时，而微调 Llama3 最少只需要5个小时（取决于微调数据量）。
    

那么我们也需要像预训练一样在微调阶段去调整所有的模型参数吗？我们来看一下实验中使用的Qwen2.5-1.5B-Instruct 的模型结构：

<style>
    table {
      width: 80%;
      margin: 20px; /* Center the table */
      border-collapse: collapse; /* Collapse borders for a cleaner look */
      font-family: sans-serif; 
    }

    th, td {
      padding: 10px;
      text-align: left;
      border: 1px solid #ddd; /* Light gray border */
    }

    th {
      background-color: #f2f2f2; /* Light gray background for header */
      font-weight: bold;
    }

    tr:nth-child(even) { /* Zebra striping */
      background-color: #f9f9f9;
    }

    tr:hover { /* Highlight row on hover */
      background-color: #e0f2ff; /* Light blue */
    }
</style>
<table width="80%">
<tbody>
<tr>
<td>

```shell
Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 1536)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (input_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
        (self_attn): Qwen2SdpaAttention(
          (q_proj): Linear(in_features=1536, out_features=1536, bias=True)
          (k_proj): Linear(in_features=1536, out_features=256, bias=True)
          (v_proj): Linear(in_features=1536, out_features=256, bias=True)
          (o_proj): Linear(in_features=1536, out_features=1536, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (post_attention_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (up_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (down_proj): Linear(in_features=8960, out_features=1536, bias=False)
          (act_fn): SiLU()
        )        
      )
    )
    (norm): Qwen2RMSNorm((1536,), eps=1e-06)
  )
  (lm_head): Linear(in_features=1536, out_features=151936, bias=False)
)
```
</td>
</tr>
</tbody>
</table>

这里简要介绍一下模型结构：

1.  `Embedding(151936, 1536)`：模型的输入嵌入层（Embedding），每个151936维 token 被映射到一个1536维的向量空间中。我们可以知道该模型的词汇表包含 151936 个词汇。
    
2.  `28 x Qwen2DecoderLayer`: 模型的核心部分，基于 Transformer 的解码器结构。它由28层相同的解码器层（`Qwen2DecoderLayer`）组成。每一层包含：
    
    1.  `input_layernorm`：Layer Normalization 层, 对注意力层输入进行归一化，使用的是RMSNorm（均方根层归一化），参数eps设为1e-06以避免除零错误。
        
    2.  `Qwen2SdpaAttention`: 自注意力机制 + 多头注意力机制 + 旋转位置编码。
        
    3.  `post_attention_layernorm`: 对注意力层进行归一化。
        
    4.  `Qwen2MLP`: 带门控的MLP （MultiLayer Perceptron）层。
        
3.  输出时的`Qwen2RMSNorm`: 在所有解码器层之后，还有一个RMSNorm层用于最终的输出归一化。
    
4.  `lm_head, Linear`: 模型的最后一层，一个线性变换层，将输出尺寸从1536维变换回词汇表大小（151,936），用于预测输出 token。
    

从调整参数量的大小这个角度，我们可以把微调分为**全参微调**与**高效微调**。

### 2.1. 全参微调
全参微调是在预训练模型的基础上进行**全量参数**微调的模型优化方法，也就是在上边的模型结构中，只要有参数，就会被调整。该方法避免消耗重新开始训练模型所有参数所需的大量计算资源，又能避免部分参数未被微调导致模型性能下降。但是，大模型训练成本高昂，需要庞大的计算资源和大量的数据，即使是全参数微调，往往也需要较高的训练成本。


### 2.2. 高效微调
因此业界研究出多种不同的**高效微调 PEFT**（Parameter-Efficient Fine-Tuning）技术，旨在降低微调参数的数量和计算复杂度的同时提高预训练模型在新任务上的性能，缓解微调大型预训练模型的训练成本，在上边的模型结构中，只需要调整某几层网络的参数即可。**高效微调 PEFT**技术的推广使得很多技术团队、研究人员可以在有限计算资源上完成模型训练，让微调大模型的路径更易获得。当前比较主流的 PEFT 方法包括 Adapter Tuning、Prompt Tuning、P-Tuning、LoRA、QLoRA 等等。

我们将简要介绍几种高效微调的方法，向你展示**高效微调**怎么能够在减少训练参数量的同时，达到与全参微调相似的效果。（调整结构，突出层次）

#### Adapter Tuning

**原理简述**

该方法会在原有的模型架构上，在某些位置之间插入Adapter层，微调训练时只训练这些Adapter层，而原先的参数不会参与训练。该方法可以在只额外增加3.6%参数的情况下达到与全参数微调相似的效果。原理图如下所示：

<div align="center">
<img src="https://alidocs.oss-cn-zhangjiakou.aliyuncs.com/res/oJGq765B3YDrnAKe/img/2e8a54f8-12e7-4de0-8271-f5b4585a457e.png" alt="adapter原理" width="600px">
</div>

假设你经营着一家饮料公司，你原本销售的方法很简单，就是把产品装箱后堆放在工厂门口，购买者只能来工厂门口才能买到，这限制了饮料的售卖渠道。于是你增加了一个物流装箱的包装工作站，只需要增加几个人，就可以把产品打包到运输箱里，从而交给快递公司发走。虽然打包的过程增加了3.6%的人力成本（类比于插入Adapter层引入的新参数量），但是现在可以向外地发货，并且原有的工作流程没有改变。这个新增的包装工作站，就类比于Adapter的作用，包装工作站中新增的人员、管理制度，就是我们需要微调的参数，而生产线等其它部门的人员和制度不用做任何调整。

#### LoRA（Low-Rank Adaptation）

LoRA是一种高效的参数高效微调方法，它通过引入一组低秩矩阵来调整预训练模型的权重，而不是直接更新所有参数。这种方法能够在不显著增加参数量的情况下，有效地对模型进行微调，特别适合于资源有限或需要快速迭代的场景。LoRA的核心思想是，对于每一个要调整的权重矩阵`D`，它不是直接修改`D`，而是添加一个低秩矩阵`A * B`（其中`A`和`B`的维度远小于`D`），这样既保留了原始模型的泛化能力，又实现了对特定任务的适应性增强。

<div align="center">
<img src="https://alidocs.oss-cn-zhangjiakou.aliyuncs.com/res/oJGq765B3YDrnAKe/img/b2e2caf3-e8da-4ac8-b57d-abbb05f932c4.png" alt="lora原理" width="800px">
</div>

如果你对秩和矩阵不熟悉的话也没有关系，你只需了解到LoRA微调是一种效果与全参微调接近，并且最通用的高效微调方法之一。如果你对LoRA的更多细节感兴趣，可以阅读：

[_LoRA: Low-Rank Adaptation of Large Language Models_](https://arxiv.org/abs/2106.09685)

现在**最常用的微调方法就是LoRA及其相关衍生技术**,因为它相对于其他微调方法有非常多的优点。首先它不破坏预训练模型结构,使得模型不会丢失预训练时获得的知识。其次,它的实现非常方便,易于工程人员在快速进行实现和集成。最后它的回退也非常方便,微调的成果只存储在"微调参数矩阵"中,与预训练模型的机构和参数可以完全分离。本文也将以lora微调作为实验案例。

## 🧬 3. 微调数据集的构建

### 3.1. 微调数据格式

<style>
    table {
      width: 80%;
      margin: 20px; /* Center the table */
      border-collapse: collapse; /* Collapse borders for a cleaner look */
      font-family: sans-serif; 
    }

    th, td {
      padding: 10px;
      text-align: left;
      border: 1px solid #ddd; /* Light gray border */
    }

    th {
      background-color: #f2f2f2; /* Light gray background for header */
      font-weight: bold;
    }

    tr:nth-child(even) { /* Zebra striping */
      background-color: #f9f9f9;
    }

    tr:hover { /* Highlight row on hover */
      background-color: #e0f2ff; /* Light blue */
    }
</style>
<table width="80%">
<tbody>
<tr>
<td>

```json
{"messages": [
  {"role": "system", "content": "<系统输入1>"}, 
  {"role": "user", "content": "<用户输入1>"}, 
  {"role": "assistant", "content": "<模型期望输出1>"}, 
  {"role": "user", "content": "<用户输入2>"}, 
  {"role": "assistant", "content": "<模型期望输出2>"}]}
```
</td>
</tr>
</tbody>
</table>

### 3.2. 数据集的规模要求

对于模型微调来说，数据集最少需要**1000+条优质微调数据**。如果数据调优后的模型评测结果不佳，最简单的改进方法是收集更多数据进行训练。

如果你缺乏数据，建议构建 RAG 检索增强应用，使用知识库索引来增强模型能力。当然在很多复杂的业务场景，可以综合采用模型调优和知识库检索结合的技术方案。

以客服场景为例，可以借助模型调优可以解决客服回答的语气、表达习惯、自我认知等问题，场景涉及的专业知识可以结合知识库，动态引入到模型上下文中。

本课程推荐你可以先构建 RAG 应用试运行，在收集到足够的应用数据后再通过模型调优继续提升模型表现。

你也可以采用以下策略扩充数据集：

1.  让大模型模拟生成特定业务/场景的相关内容，辅助你生成更多用于微调数据。（生成模型建议选取表现优异、规模更大的模型）
    
2.  通过应用场景收集、网络爬虫、社交媒体和在线论坛、公开数据集、合作伙伴与行业资源、用户贡献等各种方式，人工获取更多数据。


### 3.3. 构建高效微调数据集的策略

*   **数据质量**：确保数据集的准确性和相关性，避免模糊和错误内容。
    
*   **数据多样性**：覆盖任务的所有关键方面和潜在变化，包括不同场景、语境和专业术语。
    
*   **平衡性**：如果任务涉及多种类别场景，确保各类别样本均衡，防止模型偏向于某一类。
    
*   **持续迭代**：微调是一个迭代过程，根据模型在验证集上的表现反馈，不断优化和扩大数据集。
    
### 3.4. 训练集、验证集、测试集拆分

微调数据集分为训练集，验证集和测试集，比例通常是80%：10%：10%。

*   **训练集**：用于模型参数的学习和更新。
    
*   **验证集**：用于**快速评估模型训练**的效果，从而调整模型的超参数（如学习率、退火策略等）和监控过拟合情况。(ms-swift框架会自动从训练集中切分出验证集)
    
*   **测试集**：用于评估模型训练的效果。本课程提供的实验数据中，测试集涉及的数据库均没有在训练、验证数据集中出现过。
    
#### 注意数据长度限制

字符与 token 的对应关系请参考[字符串与Token之间的互相转换](https://help.aliyun.com/zh/model-studio/billing-for-model-studio#f79637c5f7dcl)。

如果单条数据 token 长度超过模型可以接受的最大值，微调架构可能会报错、丢弃、截短该数据。本课程使用的魔塔社区开源微调框架将会直接丢弃超过长度的单条数据。


## 🛠️ 4. 一个完整的微调实践

本课程使用Qwen2.5 1.5b 模型 +  LoRA 微调，在本地实验环境 / 魔塔开源社区免费实验实例对 NL2SQL 任务进行微调。 环境仅包含一个 16核 CPU + 32 GB 内存，没有 GPU。

NL2SQL：大模型根据数据库情况，将用户的自然语言转换成数据库查询 SQL 语句。

<div align="center">
<img src="https://alidocs.oss-cn-zhangjiakou.aliyuncs.com/res/oJGq765B3YDrnAKe/img/92c133a5-3ba1-4f97-b8f5-2d98de7843e2.png" alt="sql语句" width="600px">
</div>

本课程使用 ms-swift 框架， [ms-swift](https://github.com/modelscope/ms-swift/tree/main) （Modelscope Scalable lightWeight Infrastructure for Fine-Tuning）是魔塔社区专门为模型训练开发的开源框架，该框架支持350+ LLM和90+ MLLM（多模态大模型）的训练(预训练、微调、对齐)、推理、评测和部署。开发者可以直接将我们的框架应用到自己的Research和生产环境中，实现模型训练评测到应用的完整链路。我们除支持了[PEFT](https://github.com/huggingface/peft)提供的轻量训练方案外，也提供了一个完整的Adapters库以支持最新的训练技术，如NEFTune、LoRA+、LLaMA-PRO等。

本课程将提供 Python 和 Shell 两种方式调用 ms-swift 框架进行微调。 


### 4.1. 实验环境准备

#### 魔塔开源社区免费实验实例

请注册并登录魔塔开源社区，前往[我的 Notebook](https://modelscope.cn/my/mynotebook/preset)。启动魔塔平台免费实例。（本次实验为 CPU 环境）

等待实例启动后点击**查看 Notebook**，进入环境。

<div align="center">
<img src="https://alidocs.oss-cn-zhangjiakou.aliyuncs.com/res/oJGq765B3YDrnAKe/img/d03feb35-92e9-4809-9a08-04e8a525a5f4.png" alt="魔搭社区" width="800px">
</div>

在实验环境上更新所需包。
<style>
    table {
      width: 80%;
      margin: 20px; /* Center the table */
      border-collapse: collapse; /* Collapse borders for a cleaner look */
      font-family: sans-serif; 
    }

    th, td {
      padding: 10px;
      text-align: left;
      border: 1px solid #ddd; /* Light gray border */
    }

    th {
      background-color: #f2f2f2; /* Light gray background for header */
      font-weight: bold;
    }

    tr:nth-child(even) { /* Zebra striping */
      background-color: #f9f9f9;
    }

    tr:hover { /* Highlight row on hover */
      background-color: #e0f2ff; /* Light blue */
    }
</style>

<table width="80%">
<tbody>
<tr>
<td>

```shell
pip install --upgrade pip
pip install 'ms-swift[llm]'==2.4.2.post2
pip install evalscope==0.5.5rc1
pip install rouge_score==0.1.2
```
</td>
</tr>
</tbody>
</table>

#### 本地实验环境

建议安装 [Miniconda](https://docs.anaconda.com/miniconda/)。Miniconda 是 Anaconda 的轻量化版本，它为使用者提供了一个易于安装、管理和使用的独立 python 虚拟环境。

安装好 Miniconda 后，创建并进入虚拟环境：
<table width="80%">
<tbody>
<tr>
<td>

```shell
conda create -n finetune python=3.10
conda activate finetune
```
</td>
</tr>
</tbody>
</table>
该课程发布时，ms-swift 框架仅支持 python 3.10 环境。如需获取实时支持信息，请参见**魔塔开源社区免费实验实例**的环境配置。

<div align="center">
<img src="https://alidocs.oss-cn-zhangjiakou.aliyuncs.com/res/oJGq765B3YDrnAKe/img/cc1c6056-37d0-4a70-a58a-8fd1f98cf748.png" alt="notebook" width="800px">
</div>

在本地的虚拟环境安装所需包。

<table width="80%">
<tbody>
<tr>
<td>

```shell
pip install --upgrade pip
pip install 'ms-swift[llm]'==2.4.2.post2
pip install evalscope==0.5.5rc1
```
</td>
</tr>
</tbody>
</table>


### 4.2. 把模型下载到本地（可选）

<table width="80%">
<tbody>
<tr>
<td>

```shell
modelscope download --model qwen/Qwen2.5-1.5B-Instruct --local_dir '<模型本地位置>'
```
</td>
</tr>
</tbody>
</table>

### 4.3. 准备微调数据集

这里提供线上教育公司相关数据库查询问题，其中大概 500+ 作为训练集，100+ 作为测试集。

请将此压缩包的内容上传并解压到实验环境。


#### 数据格式
<table width="80%">
<tbody>
<tr>
<td>

```json
{"messages": [
  {"role": "system", "content": "<系统输入1>"}, 
  {"role": "user", "content": "<用户输入1>"}, 
  {"role": "assistant", "content": "<模型期望输出1>"}, 
  {"role": "user", "content": "<用户输入2>"}, 
  {"role": "assistant", "content": "<模型期望输出2>"}]}
```
</td>
</tr>
</tbody>
</table>

In [3]:
# 下载数据示例文件
from IPython.display import display, FileLink

local_file = FileLink('resources/2_4/data.zip', result_html_prefix="下载实验数据集 ")
display(local_file)

/Users/lekayef/Desktop/scripts/ACP 课程模型微调/aliyun_acp_learning/大模型ACP认证教程/p2_构造大模型问答系统/resources/2_4/data.zip

解压后包含文件

<img src="https://alidocs.oss-cn-zhangjiakou.aliyuncs.com/res/oJGq765B3YDrnAKe/img/67e3aae7-7340-416e-a60c-f043ffc1fe53.png" alt="解压包文件" width="300px">

train.jsonl 为训练集文件。test.jsonl 为测试集文件，config\_eval.json 为 ms-swift 评估自定义数据集所需的配置。

config\_eval.json 内容：

<img src="https://alidocs.oss-cn-zhangjiakou.aliyuncs.com/res/oJGq765B3YDrnAKe/img/1349a6b2-844e-4bbb-ae27-7dd8f713371f.png" alt="eval.json 内容" width="640px">


**评测前需要手动修改**`dataset`**的路径。**

### 4.4. 使用预训练模型
> 如果将模型没有下载到本地，请删除 --model\_id\_or\_path 参数
<style>
    table {
      width: 80%;
      margin: 20px; /* Center the table */
      border-collapse: collapse; /* Collapse borders for a cleaner look */
      font-family: sans-serif; 
    }

    th, td {
      padding: 10px;
      text-align: left;
      border: 1px solid #ddd; /* Light gray border */
    }

    th {
      background-color: #f2f2f2; /* Light gray background for header */
      font-weight: bold;
    }

    tr:nth-child(even) { /* Zebra striping */
      background-color: #f9f9f9;
    }

    tr:hover { /* Highlight row on hover */
      background-color: #e0f2ff; /* Light blue */
    }
</style>
<table width="80%">
<tbody>
<tr>
<td>

```shell
swift infer \
--model_id_or_path '<模型本地位置>' \
--model_type 'qwen2_5-1_5b-instruct'
```
</td>
</tr>
</tbody>
</table>
模型使用时支持设置常用的“top-p”、“top-k”、“temperature”等参数。

完整参数列表请参见：[infer 命令行参数](https://swift.readthedocs.io/zh-cn/latest/Instruction/%E5%91%BD%E4%BB%A4%E8%A1%8C%E5%8F%82%E6%95%B0.html#infer-merge-lora)

### 4.5. 评测预训练模型性能

评测模式为：general\_qa。相关信息请参见：[问答题格式数据集构建方法](https://evalscope.readthedocs.io/zh-cn/latest/advanced_guides/custom_dataset.html#qa)

评测命令：

> 如果将模型没有下载到本地，请删除 --model\_id\_or\_path 参数

<table width="80%">
<tbody>
<tr>
<td>

```shell
swift eval \
--model_id_or_path '<模型本地位置>' \
--model_type 'qwen2_5-1_5b-instruct' \
--eval_dataset no \
--custom_eval_config '<微调数据集压缩包中config_eval.json相对或绝对位置>' \
--max_length -1 \
--name 'pre_train_evaluation' \
--eval_output_dir './eval_outputs'
```
</td>
</tr>
</tbody>
</table>
完整参数列表请参见：[eval 命令行参数](https://swift.readthedocs.io/zh-cn/latest/Instruction/%E5%91%BD%E4%BB%A4%E8%A1%8C%E5%8F%82%E6%95%B0.html#eval)

关键运行结果：

<img src="https://alidocs.oss-cn-zhangjiakou.aliyuncs.com/res/oJGq765B3YDrnAKe/img/1324f2b0-5c49-44bc-87f0-49ad194e3007.png" alt="运行结果" width="640px">

这里简单解释一下各评测指标：

1.  **ROUGE Scores (Recall-Oriented Understudy for Gisting Evaluation)**:
    
    *   **rouge-n-(r/p/f)**: 这是基于词级别的n-gram匹配度量，其中 n=1，意味着考虑单个词汇的重合情况； n=2，意味着考虑两个词汇组成的短语的重合情况；n=l, 意味着考虑一个模型输出与测试用例中最长公共子序列的重合情况。
        
        *   **rouge-n-r**: 召回率（Recall），表示模型输出正确预测出测试用例的比例。
            
        *   **rouge-n-p**: 精确率（Precision），表示模型输出有多大部分也出现在测试用例中。
            
        *   **rouge-n-f**: F1分数，是召回率和精确率的调和平均值，综合考量了两者的表现。
            

> 相关文章：ROUGE: A Package for Automatic Evaluation of Summaries (Chin-Yew Lin, 2004)

2.  **BLEU Scores (Bilingual evaluation understudy)**:
    
    *   **bleu-1/2/3/4**: 这些是基于n-gram的匹配度量，用于评估机器翻译或其他文本生成任务的质量。
        
        *   **bleu-1**: 基于单个词的匹配情况。
            
        *   **bleu-2**: 基于双词（bigram）的匹配情况。
            
        *   **bleu-3**: 基于三词（trigram）的匹配情况。
            
        *   **bleu-4**: 基于四词（4-gram）的匹配情况。
            

> 相关文章：BLEU: a Method for Automatic Evaluation of Machine Translation (Kishore Papineni, 2002)

总体来说，本次实验要求输出正确的 SQL 语句，那 `rouge-l`的 `r/p/f`指标应该作为模型评测的参考标准，指标越接近1，说明生成 SQL 语句有用部分越多，模型表现越好。


###  4.6. 微调参数说明

微调命令：

> 如果将模型没有下载到本地，请删除 --model\_id\_or\_path 参数
<table width="80%">
<tbody>
<tr>
<td>

```shell
swift sft \
--dataset '<微调数据集压缩包中train.json相对或绝对位置>' \
--learning_rate '1e-4' \
--eval_steps '10' \
--save_steps '10' \
--batch_size '8' \
--model_type 'qwen2_5-1_5b-instruct' \
--max_length -1 \
--model_id_or_path '/root/model/' \
--num_train_epochs 1
```
</td>
</tr>
</tbody>
</table>
完整参数列表请参见：[sft 命令行参数](https://swift.readthedocs.io/zh-cn/latest/Instruction/%E5%91%BD%E4%BB%A4%E8%A1%8C%E5%8F%82%E6%95%B0.html#sft-%E5%8F%82%E6%95%B0) 

*   在模型微调时，框架会把训练数据以8:2的比例分成训练集和验证集，并在微调时根据参数设置在模型训练完成一定条件后自动进行交叉验证，计算验证集的 loss 和准确度。  
    例如：实验中设置`--eval_steps ‘10‘`，那每训练10个 step，框架进行一次交叉验证，输出验证集的 loss 和准确度。
    
*   框架在微调时默认使用 LoRA 高效微调方法，在命令中不需要额外声明。如果想修改微调方式请声明参数`--sft_type`。
    <img src="https://alidocs.oss-cn-zhangjiakou.aliyuncs.com/res/oJGq765B3YDrnAKe/img/8a008878-edba-48b4-9d7e-3e58b2154890.png" alt="参数sfttype" width="640px">
   
    与自动交叉验证类似，通过设置`--save_steps '10'`，每训练10个 step，框架会自动把训练获得的“微调参数矩阵”保存在`output/qwen2_5-1_5b-instruct/`下。并且在训练结束时，并不会把**“微调参数矩阵”**与**“预训练权重矩阵”**合并。  
    <div align="center">
    <img src="https://alidocs.oss-cn-zhangjiakou.aliyuncs.com/res/oJGq765B3YDrnAKe/img/b2e2caf3-e8da-4ac8-b57d-abbb05f932c4.png" alt="lora原理" width="800px">
    </div>
    框架会保留训练时产生的**最近两次**“微调参数矩阵”。如果当你观察到模型训练出现过拟合等现象，需要终止训练，那终止训练也不会浪费宝贵的时间和资源，因为阶段性的训练成果都保存在“微调参数矩阵”里。
    
*   如果想基于已经微调后的模型继续进行训练，可以先融合“微调参数矩阵”与“预训练权重矩阵”，得到微调后的模型全参，再在微调命令时使用`--model_id_or_path`参数，设定参数值为全参路径。
    

这里简要介绍一下模型训练以及 LoRA 高效微调涉及的常用参数。

|  参数名称  |  默认设置  |  参数作用  |
| --- | --- | --- |
|  \--num\_train\_epochs  |  1  |  模型遍历数据集的次数。  |
|  \--learning\_rate  |  1e-4  |  模型训练的学习率，用于控制模型修正权重的强度。 如果学习率设置得太高，模型参数会剧烈变化，导致微调后的模型表现不一定更好，甚至变差； 如果学习率太低，微调后的模型表现不会有太大变化。  |
|  \--batch\_size  |  1  |  一次性送入模型进行训练的数据条数。参数过大可能导致训练时内存不足，训练失败。  |
|  \--eval\_steps  |  50  |  训练阶段针对模型的验证间隔步长，用于阶段性评估模型训练效果。  |
|  \--max\_length  |  2048  |  指的是单条训练数据 token 支持的最大长度。如果单条数据 token 长度超过设定值, ms-swift 框架将丢弃该条数据。 字符与 token 之间的关系请参考 [Token和字符串之间怎么换算](https://help.aliyun.com/zh/model-studio/billing-for-model-studio#35d3b2ad2386c)  |
|  \--target\_modules  |  'DEFAULT'  |  LoRA 的矩阵分解目标模块仅为`Qwen2SdpaAttention`  |
|  \--lora\_rank  |  8  |  LoRA训练中的低秩矩阵的秩大小。秩越大调优效果会更好一点，但训练会略慢。  |
|  \--lora\_alpha  |  32  |  用于控制“预训练权重矩阵”与“微调参数矩阵”的相加系数。 相当于：全参矩阵 = 预训练权重矩阵 + lora\_alpha \* 微调参数矩阵 该参数的作用类似于`--learning_rate`。  |
|  \--lora\_dropout  |  0.05  |  训练时“预训练权重矩阵”值的丢弃率。用于抑制模型过拟合。  |



### 4.7. 快速估模型训练的效果

如前面章节所说，验证集的作用是用于快速评估模型训练的效果。

那如何评估模型训练的效果呢？你需要着眼于两个数值，训练损失和验证损失。

*   **如果训练损失在减少，验证损失也在减少**，那说明模型训练还没完成，还没有完全学习完训练集的内容，模型需要更多的训练来学习数据中的潜在模式。最简单的解决方法是增加`--num_train_epochs`。  
    这里可以理解为模型还是个学渣，需要多刷几遍题库才能获得更好的成绩。
    
*   **如果训练损失在减少，验证损失却在增加，**那说明模型训练完成过头了，模型已经处于**过拟合**状态，过度拟合了数据中的潜在模式。相当于模型记住了整个题库的答案，但满脑子只有题库的内容，当面对来自评测集的新题时，反而不会答题了。在 LoRA 微调中，可以通过增加`--lora_dropout`的值，抑制模型的**过拟合**倾向。
    
*   如果训练损失不减少，验证损失也不减少，（不常见）那说明模型训练失败了，需要寻找失败的原因。  
    常见的可能原因有：
    
    *   学习率设置过大，导致模型参数剧烈震荡。
        
    *   训练数据有问题，例如模型输入和预期输出不存在任何逻辑关系。
        
    *   抑制模型过拟合的参数设置过大，导致模型无法学习新知识。
        

### 4.8. 使用微调后模型
<style>
    table {
      width: 80%;
      margin: 20px; /* Center the table */
      border-collapse: collapse; /* Collapse borders for a cleaner look */
      font-family: sans-serif; 
    }

    th, td {
      padding: 10px;
      text-align: left;
      border: 1px solid #ddd; /* Light gray border */
    }

    th {
      background-color: #f2f2f2; /* Light gray background for header */
      font-weight: bold;
    }

    tr:nth-child(even) { /* Zebra striping */
      background-color: #f9f9f9;
    }

    tr:hover { /* Highlight row on hover */
      background-color: #e0f2ff; /* Light blue */
    }
</style>
<table width="80%">
<tbody>
<tr>
<td>

```shell
swift infer \
--ckpt_dir 'lora微调后checkpoint相对或绝对位置'
```
</td>
</tr>
</tbody>
</table>
模型训练完成后，会把“微调参数矩阵”（checkpoint）和相关日志存储在`output`目录下。

模型使用时支持设置常用的“top-p”、“top-k”、“temperature”等参数。

完整参数列表请参见：[infer 命令行参数](https://swift.readthedocs.io/zh-cn/latest/Instruction/%E5%91%BD%E4%BB%A4%E8%A1%8C%E5%8F%82%E6%95%B0.html#infer-merge-lora)

### 4.9. 参数矩阵融合（可选）

模型微调训练完成后，我们有两种方式发布和使用微调后的模型，一种方式是直接融合基础模型和微调模型的参数矩阵得到融合模型，发布或加载融合模型；另一种方式是不做预融合，仅在调用时动态加载微调模型。

假设，原Qwen1.5B的模型有15亿参数，微调模型有2百万参数。那么，融合后的模型仍然有15亿参数，并且新模型与原Qwen1.5B参数大小相同，占存储空间大小相同。这种方式适合做整合好的模型版本发布。

如果不进行模型融合，微调产生的模型可能只有2百万参数，假设微调模型参数矩阵只占100兆字节存储空间，这个大小非常便于做增量发布和传播。这也是我们工程上常用的方法。需要注意的是，用哪个基座模型微调，我们在使用时就需要指定使用哪个基座模型，再动态加载微调模型。

这里我们介绍融合“微调参数矩阵”与“预训练权重矩阵”的方法，将微调后的模型参数存储成一个完整的参数矩阵。
<table width="80%">
<tbody>
<tr>
<td>

```shell
swift export \
--ckpt_dir 'lora微调后checkpoint相对或绝对位置' \
--merge_lora true
```
</td>
</tr>
</tbody>
</table>
融合后的矩阵默认存储在checkpoint目录下。

### 4.10. 评测微调后的模型
<style>
    table {
      width: 80%;
      margin: 20px; /* Center the table */
      border-collapse: collapse; /* Collapse borders for a cleaner look */
      font-family: sans-serif; 
    }

    th, td {
      padding: 10px;
      text-align: left;
      border: 1px solid #ddd; /* Light gray border */
    }

    th {
      background-color: #f2f2f2; /* Light gray background for header */
      font-weight: bold;
    }

    tr:nth-child(even) { /* Zebra striping */
      background-color: #f9f9f9;
    }

    tr:hover { /* Highlight row on hover */
      background-color: #e0f2ff; /* Light blue */
    }
</style>
<table width="80%">
<tbody>
<tr>
<td>

```shell
swift eval \
--ckpt_dir 'lora微调后checkpoint相对或绝对位置' \
--eval_dataset no \
--custom_eval_config '<微调数据集压缩包中config_eval.json相对或绝对位置>' \
--max_length -1 \
--name 'finetune_evaluation' \
--system '' \
--eval_output_dir './eval_outputs'
```

</td>
</tr>
</tbody>
</table>

完整参数列表请参见：[eval 命令行参数](https://swift.readthedocs.io/zh-cn/latest/Instruction/%E5%91%BD%E4%BB%A4%E8%A1%8C%E5%8F%82%E6%95%B0.html#eval)

|  预训练模型评测结果  |  模型训练一轮后评测结果 \> \--num\_train\_epochs 1  |  模型训练三轮后评测结果 \> \--num\_train\_epochs 3  |
| --- | --- | --- |
|  <img src="https://alidocs.oss-cn-zhangjiakou.aliyuncs.com/res/oJGq765B3YDrnAKe/img/3d38356f-c8e2-46aa-8df7-a429dcec7562.png" alt="预训练模型评测结果" width="380px"> | <img src="https://alidocs.oss-cn-zhangjiakou.aliyuncs.com/res/oJGq765B3YDrnAKe/img/9589431d-922c-4d65-b3f7-b71cc1b06d98.png" alt="模型训练一轮后评测结果" width="360px"> | <img src="https://alidocs.oss-cn-zhangjiakou.aliyuncs.com/res/oJGq765B3YDrnAKe/img/f8b6a837-725f-47ac-a22e-e3701b6ce14f.png" alt="模型训练三轮后评测结果" width="360px">  |


让我们来看看模型训练后的实际表现。模型输入：
<table width="80%">
<tbody>
<tr>
<td>

```markdown
#背景#
数据库信息:{'column_names': [[-1, '*', 'text'], [0, 'region id', 'number'], [0, 'region name', 'text'], [1, 'country id', 'text'], [1, 'country name', 'text'], [1, 'region id', 'number'], [2, 'department id', 'number'], [2, 'department name', 'text'], [2, 'manager id', 'number'], [2, 'location id', 'number'], [3, 'job id', 'text'], [3, 'job title', 'text'], [3, 'min salary', 'number'], [3, 'max salary', 'number'], [4, 'employee id', 'number'], [4, 'first name', 'text'], [4, 'last name', 'text'], [4, 'email', 'text'], [4, 'phone number', 'text'], [4, 'hire date', 'time'], [4, 'job id', 'text'], [4, 'salary', 'number'], [4, 'commission pct', 'number'], [4, 'manager id', 'number'], [4, 'department id', 'number'], [5, 'employee id', 'number'], [5, 'start date', 'time'], [5, 'end date', 'time'], [5, 'job id', 'text'], [5, 'department id', 'number'], [6, 'location id', 'number'], [6, 'street address', 'text'], [6, 'postal code', 'text'], [6, 'city', 'text'], [6, 'state province', 'text'], [6, 'country id', 'text']], 'foreign_keys': [[5, 1], [20, 10], [24, 6], [28, 10], [29, 6], [25, 14], [35, 3]], 'primary_keys': [1, 3, 6, 10, 14, 25, 30], 'table_names': ['regions', 'countries', 'departments', 'jobs', 'employees', 'job history', 'locations']}
#受众#
Mysql数据库
#输出#
只输出SQL查询语句
#目的#
将问题\"What is all the information about employees who have never had a job in the past?\"转换为转化为SQL查询语句
```

</td>
</tr>
</tbody>
</table>

这里简单解释一下这里的数据库信息字典的含义：

`column_names`的一个元素`[1, 'country id', 'text']`表示：数据库内第二张表有一个名叫`'country id'`的列，该列属性为`'text'`。

`foreign_keys`和`primary_keys`中的数字，指的是`column_names`中的列位置，比如数据库中第一张表的`primary_keys`是`'region id'`。

`table_names`自然指的是该数据库有哪些表。

期望输出：
<table width="80%">
<tbody>
<tr>
<td>

```sql
SELECT * FROM employees WHERE employee_id NOT IN (SELECT employee_id FROM job_history)
```
</td>
</tr>
</tbody>
</table>
预训练模型输出：
<table width="80%">
<tbody>
<tr>
<td>

```sql
SELECT *
FROM employees
WHERE employee_id NOT IN (
    SELECT e.employee_id
    FROM employees e
    INNER JOIN job_history jh ON e.employee_id = jh.employee_id
)
```

</td>
</tr>
</tbody>
</table>

微调后模型输出：

> \--num\_train\_epochs 3

```sql
SELECT * FROM employees WHERE employee_id NOT IN ( SELECT DISTINCT employee_id FROM job_history )
```

我们可以看到微调后模型的表现确实如评测结果显示的一样，在经过微调后在测试集的问题上表现得比预训练模型要更好。

## ✅ 本节小结
通过学习本节课程，你已经掌握了模型微调的基本知识并进行了简单实践。下一节你将学习如何使用插件和智能体继续增强大模型能力。

## 🔥 课后小测验

【单选题】2.4.1. 以下关于 LoRA 的描述，哪一项是错误的？（ ）

A. LoRA 可以有效降低微调大型语言模型的成本。

B. LoRA 会修改预训练模型的原始权重。

C. LoRA 的实现相对简单，易于集成。

D. LoRA 微调的结果可以方便地回退。

答案：B

解析： LoRA 并不直接修改原始权重，而是通过添加低秩矩阵间接影响模型行为。 这使得回退操作变得简单，只需移除添加的低秩矩阵即可。

<br>

【多选题】2.4.2. 你正在使用 Swift 微调一个 Qwen 模型，发现模型在验证集上的 loss 出现了明显的上升趋势，以下哪些操作可以帮助你缓解或解决这个问题？( )

A. 增大 --learning_rate

B. 减小 --learning_rate

C. 增大 --lora_dropout

D. 减小 --lora_dropout

E. 增大 --num_train_epochs

F. 减小 --num_train_epochs

答案: B, C, F

解释:

- learning_rate: 学习率过大会导致模型训练速度快，但可能在最优解附近震荡，甚至无法收敛，导致loss波动，看起来像是过拟合。当然这与真正的过拟合不同。
- lora_dropout: 增大 dropout 率可以增强模型的泛化能力，抑制过拟合。
- num_train_epochs: 过拟合也可能是训练次数过多导致的，减少训练次数可以避免模型过度学习训练数据。